### Go Through The Code

In [1]:
from scapy.all import rdpcap, IP, TCP, UDP, ICMP, Raw, IPv6
from collections import defaultdict

In [2]:
pcap_file = "0.pcap"
try:
    packets = rdpcap(pcap_file)
except FileNotFoundError:
    print(f"File {pcap_file} not found")

In [3]:
packet = packets[100]
packet.show()

###[ Ethernet ]###
  dst       = 00:1a:8c:15:f9:80
  src       = 40:61:86:9a:f1:f5
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 17344
     flags     = DF
     frag      = 0
     ttl       = 128
     proto     = tcp
     chksum    = 0x526a
     src       = 192.168.3.131
     dst       = 65.54.95.68
     \options   \
###[ TCP ]###
        sport     = 56511
        dport     = http
        seq       = 3457485297
        ack       = 1036685706
        dataofs   = 5
        reserved  = 0
        flags     = A
        window    = 16425
        chksum    = 0x869a
        urgptr    = 0
        options   = []



In [26]:
protocols = defaultdict(int)
count = 0
for packet in packets:
    if IP in packet:
        protocols[packet[IP].proto] += 1
        # if packet[IP].proto == 0 and count == 0:
        #     icmp_packet = packet
        #     count += 1
    if IPv6 in packet:
        protocols[packet[IPv6].nh] += 1

In [25]:
igmp_packet.show()

###[ Ethernet ]###
  dst       = 00:90:7f:3e:02:d0
  src       = 30:e4:db:b1:58:60
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 64
     id        = 30923
     flags     = 
     frag      = 0
     ttl       = 255
     proto     = icmp
     chksum    = 0xe3e8
     src       = 172.16.133.233
     dst       = 172.16.128.254
     \options   \
###[ ICMP ]###
        type      = echo-request
        code      = 0
        chksum    = 0x5d98
        id        = 0x766
        seq       = 0x4
        unused    = b''
###[ Raw ]###
           load      = b'\x00\x00\x00\x00\x13\x8a\x1a4\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd'



In [22]:
hop_packet.show()

###[ Ethernet ]###
  dst       = 33:33:00:00:00:16
  src       = c0:18:85:5a:f9:e2
  type      = IPv6
###[ IPv6 ]###
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 76
     nh        = Hop-by-Hop Option Header
     hlim      = 1
     src       = fe80::3d1c:9cea:dce1:314b
     dst       = ff02::16
###[ IPv6 Extension Header - Hop-by-Hop Options Header ]###
        nh        = ICMPv6
        len       = 0
        autopad   = On
        \options   \
         |###[ Router Alert ]###
         |  otype     = Router Alert [00: skip, 0: Don't change en-route]
         |  optlen    = 2
         |  value     = Datagram contains a MLD message
         |###[ PadN ]###
         |  otype     = PadN [00: skip, 0: Don't change en-route]
         |  optlen    = 0
         |  optdata   = b''
###[ MLDv2 - Multicast Listener Report ]###
           type      = MLD Report Version 2
           res       = 0
           cksum     = 0x5669
           reserved  = 0
           records_nu

In [27]:
icmp_packet.show()

###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 64
  id        = 30923
  flags     = 
  frag      = 0
  ttl       = 255
  proto     = icmp
  chksum    = 0xe3e8
  src       = 172.16.133.233
  dst       = 172.16.128.254
  \options   \
###[ ICMP ]###
     type      = echo-request
     code      = 0
     chksum    = 0x5d98
     id        = 0x766
     seq       = 0x4
     unused    = b''
###[ Raw ]###
        load      = b'\x00\x00\x00\x00\x13\x8a\x1a4\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd\xab\xcd'



Different Type of Protocols Found During the sniff

In [12]:
protocols 

defaultdict(int, {6: 647721, 17: 153636, 1: 4300, 2: 286, 0: 34})

In [7]:
packet[IP].proto

6

In [8]:
# ims_ips = ["14.139.98.79", "10.0.137.79", "64:ff9b::e8b:624f"]  # From nslookup.com and by trying 
ims_ip = "10.0.137.79"

In [9]:
port_packets, unique_conn_to_ims, all_ims_packets, ims_dst_packets = [], defaultdict(int), [], []
super_users = 0
for packet in packets:
    IPv = IP if IP in packet else IPv6 if IPv6 in packet else None
    if IPv and IPv in packet:
        src_ip = packet[IPv].src
        dst_ip = packet[IPv].dst
        
        if dst_ip == ims_ip:
            ims_dst_packets.append(packet)
            all_ims_packets.append(packet)
        
        if src_ip == ims_ip:
            all_ims_packets.append(packet)
        
        protocol = TCP if TCP in packet else UDP if UDP in packet else None
        if protocol:
            src_port = packet[protocol].sport
            dst_port = packet[protocol].dport
            unique_conn_to_ims[f"{src_ip}:{src_port} -> {dst_ip}:{dst_port}"] += 1
            
        if protocol and (packet[protocol].sport == 4321 or packet[protocol].dport == 4321):
            port_packets.append(packet)
        
        if Raw in packet:
            payload = packet[Raw].load.decode(errors='ignore')
            super_users += payload.lower().count("superuser") > 0

In [10]:
ims_dst_packets

[<Ether  dst=66:77:88:99:aa:bb src=00:11:22:33:44:55 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=1 flags= frag=0 ttl=64 proto=tcp chksum=0xd104 src=10.1.12.123 dst=10.0.137.79 |<TCP  sport=1234 dport=4321 seq=0 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0xd064 urgptr=0 |>>>,
 <Ether  dst=66:77:88:99:aa:bb src=00:11:22:33:44:55 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=1 flags= frag=0 ttl=64 proto=tcp chksum=0xd104 src=10.1.12.123 dst=10.0.137.79 |<TCP  sport=1234 dport=4321 seq=0 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0xd064 urgptr=0 |>>>,
 <Ether  dst=66:77:88:99:aa:bb src=00:11:22:33:44:55 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=1 flags= frag=0 ttl=64 proto=tcp chksum=0xd104 src=10.1.12.123 dst=10.0.137.79 |<TCP  sport=1234 dport=4321 seq=0 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0xd064 urgptr=0 |>>>,
 <Ether  dst=66:77:88:99:aa:bb src=00:11:22:33:44:55 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=1 flags= frag=0 tt

In [11]:
port_packets

[<Ether  dst=2c:41:38:1d:e9:40 src=00:90:7f:3e:02:d0 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=103 id=17453 flags= frag=0 ttl=126 proto=udp chksum=0x9996 src=172.16.128.169 dst=172.16.133.248 |<UDP  sport=4321 dport=snmp len=83 chksum=0x9082 |<SNMP  version='v1' 0x0 <ASN1_INTEGER[0]> community=<ASN1_STRING[b'public']> PDU=<SNMPget  id=0x491c2f5 <ASN1_INTEGER[76661493]> error='no_error' 0x0 <ASN1_INTEGER[0]> error_index=0x0 <ASN1_INTEGER[0]> varbindlist=[<SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.10.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.2.2.1.16.18']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>, <SNMPvarbind  oid=<ASN1_OID['.1.3.6.1.2.1.1.3.0']> value=<ASN1_NULL[0]> noSuchObject=None noSuchInstance=None endOfMibView=None |>] |> |>>>>,
 <Ether  dst=00:90:7f:3e:02:d0 src=2c:41:38:1d:e9:40 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=115 id=62980 flags= frag=

In [12]:
len(port_packets), len(ims_dst_packets), super_users

(52, 30, 69)

In [13]:
sum([len(packet) for packet in port_packets]), sum([len(packet) for packet in ims_dst_packets])

(2970, 1620)

In [14]:
port_packets[8].show()

###[ Ethernet ]###
  dst       = 66:77:88:99:aa:bb
  src       = 00:11:22:33:44:55
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 64
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xd0ec
     src       = 10.0.137.79
     dst       = 10.1.12.123
     \options   \
###[ TCP ]###
        sport     = 4321
        dport     = 1234
        seq       = 0
        ack       = 0
        dataofs   = 5
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = 0x5b9c
        urgptr    = 0
        options   = []
###[ Raw ]###
           load      = b'course = Embedded_system'



In [15]:
# Print Results
print(f"Q1: Unique Packets Destined to IMS server: {len(ims_dst_packets)}")
print(f"Q3: Total data transferred on port 4321: {sum(len(pkt) for pkt in port_packets)} bytes")
print(f"Q4: Total number of SuperUsers: {super_users}")

Q1: Unique Packets Destined to IMS server: 30
Q3: Total data transferred on port 4321: 2970 bytes
Q4: Total number of SuperUsers: 69


In [16]:
# Print the packet with the course information
for ims_pkt in all_ims_packets:
    if Raw in ims_pkt:
        payload = ims_pkt[Raw].load.decode(errors='ignore')
        if 'course' in payload:
            print(payload)
            break

course = Embedded_system
